#### <center> Innoscape - Telecom Paristech - Projet fil rouge </center>
# <center> Mapping Groupes de produits </center>

In [1]:
# Librairies 
import pandas as pd
import numpy as np
from scipy import stats as scs
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import re
from Preprocessing2_v2 import *
from datetime import datetime

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 60)

chemin = "../Data/"
chemin_val = "/Users/valentinphetchanpheng/Documents/MS BGD Telecom ParisTech/Projet Fil Rouge/"

##  Partie Data Preparation / Preprocessing 

### Importation des données et renommage

In [2]:
# Importation des données vendeurs
df_seller_origin = pd.read_csv(chemin + "t3_all_fr_prices_posweekly_prod_without_price.csv", sep = ",")

# Création d'un df allégé pour le travail - on ne retient que les colonnes identifiées comme utiles pour l'analyse à ce stade
columns_to_keep = ['t3_all_fr_prices_posweekly_prod.productfamily_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory4_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory3_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory2_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory1_seller',
                   't3_all_fr_prices_posweekly_prod.productseller_name',
                   't3_all_fr_prices_posweekly_prod.id_source',
                   't3_all_fr_prices_posweekly_prod.brand_seller']

df_seller = df_seller_origin[columns_to_keep]

# On raccourcit un peu les noms des colonnes en ôtant les préfixes redondants
shortened_names = map(lambda string: string.replace('t3_all_fr_prices_posweekly_prod.',''), columns_to_keep)
col_name_dict = dict(zip(columns_to_keep, shortened_names))
df_seller.rename(columns=col_name_dict, inplace=True)

In [3]:
# Importation des données références catégories Innoscape
df_ref_origin = pd.read_csv(chemin + "2018.11.21_innoscape_internal_architecture.csv")
df_ref = df_ref_origin.drop("Date", axis=1)

# On garde que la colonne correspondante à la famille de produit de référence finale
columns_to_keep = ['innoscape_ref_productfamily_seller']
df_ref = df_ref[columns_to_keep]
df_ref = df_ref.drop_duplicates()

In [4]:
# On remplace les lignes contenant des NaN par ""
df_seller = df_seller.dropna(how='all', subset=['productfamily_seller',
                                                'prodcategory4_seller',
                                                'prodcategory3_seller',
                                                'prodcategory2_seller',
                                                'prodcategory1_seller',
                                                'productseller_name'])
df_seller = df_seller.fillna("")

### Regex / Caractères Spéciaux / Minuscule / Stemming / Lemmatization

In [5]:
# Stemming / Lemmatization sur les données de référence
dico1 = {",":" ", "'":" ", ";":" ", ":":" ", "\?":" ", "!":" ", "\.":'', "  ":" ", "  ":" ",
         "\(":"","\)":"","\[":"","\]":"","-":" ","\+":"","/":"","\*":"","–":" ",
         "&":"et"}

dico2 = {'à':'a','â':'a','ä':'a',
              'é':'e','è':'e','ê':'e','ë':'e',
              'ï':'i','î':'i',
              'ô':'o','ö':'o',
              'ù':'u','û':'u',
              'ç':'c'}

start = datetime.now() 
#df_test = f_spacy_stemming(df_ref, df_ref.columns, ['ref_spacy'], dico1, dico2)
df_ref = f_treetagger(df_ref, df_ref.columns, ['ref_treetagger'], dico1, dico2)
stop = datetime.now()

execution_time = stop-start 
print("Execution time : ", execution_time)

Execution time :  0:00:01.007102


In [6]:
df_ref.head(5)

innoscape_ref_productfamily_seller  \
0                            Accessoires Gros oeuvre   
1                             Accessoires couverture   
3                                 Divers Gros oeuvre   
4                 Gouttière descente et accessoires    
5  Poudre béton chaux ciment enduit mortier ragréage   

                                      ref_treetagger  
0                              accessoire gros œuvre  
1                              accessoire couverture  
3                                  divers gros œuvre  
4                   gouttiere descente et accessoire  
5  poudre beton chaux ciment enduire mortier ragr...

In [7]:
col = ['productfamily_seller', 'prodcategory4_seller', 'prodcategory3_seller', 'prodcategory2_seller', 'prodcategory1_seller', 'productseller_name']
new_col = ['productfamily_seller_new', 'prodcategory4_seller_new', 'prodcategory3_seller_new', 'prodcategory2_seller_new', 'prodcategory1_seller_new', 'productseller_name_new']

start = datetime.now()
df_seller = f_treetagger(df_seller, col, new_col, dico1, dico2)
stop = datetime.now()

execution_time = stop-start 
print("Execution time : ", execution_time)

Execution time :  0:05:50.888570


In [8]:
df_seller.head(5)

productfamily_seller prodcategory4_seller              prodcategory3_seller  \
0   Colles à carrelage   Colles à carrelage                Colles et adhésifs   
1                                                                               
2                                            Accessoires pour lave vaisselles   
3               Mastic               Mastic                Colles et adhésifs   
4                Cires                Cires   Peintures teintures et solvants   

                      prodcategory2_seller prodcategory1_seller  \
0                            Quincaillerie            Bricolage   
1                                                                 
2                              Accessoires  Gros électroménager   
3                            Quincaillerie            Bricolage   
4  Peintures outils et traitement des murs            Bricolage   

                                  productseller_name id_source  \
0  SikaCeram Xtra   Colle à carrelage intérieur e...    AMZ_FR   
1   Ariston 3200362 scaldabagno elettrico PRO 100 H3    AMZ_FR   
2  AEG Panier À Couverts Lave vaisselle Cage et H...    AMZ_FR   
3              Colle mastic pour joints d étanchéité    AMZ_FR   
4  Sikagard Protection Sol MAT   Imperméabilisant...    AMZ_FR   

       brand_seller productfamily_seller_new prodcategory4_seller_new  \
0     SIKAFRANCESAS        colle a carrelage        colle a carrelage   
1  ARISTONTHERMOSPA                                                     
2               AEG                                                     
3     SIKAFRANCESAS                   mastic                   mastic   
4     SIKAFRANCESAS                     cire                     cire   

         prodcategory3_seller_new             prodcategory2_seller_new  \
0                colle et adhesif                        quincaillerie   
1                                                                        
2  accessoire pour lave vaisselle                           accessoire   
3                colle et adhesif                        quincaillerie   
4    peinture teinture et solvant  peinture outil et traitement du mur   

  prodcategory1_seller_new                             productseller_name_new  
0                bricolage  sikaceram xtra coller a carrelage interieur et...  
1                                     ariston scaldabagno elettrico pro heure  
2      gros electromenager  aeg panier a couvert laver vaisselle cage et h...  
3                bricolage               colle mastic pour joint d etancheite  
4                bricolage  sikagard protection sol mat impermeabiliser po...

### StopWords + Marques

In [9]:
stop_words = {'a',
             'ai',
             'aie',
             'aient',
             'aies',
             'ait',
             'as',
             'au',
             'aura',
             'aurai',
             'auraient',
             'aurais',
             'aurait',
             'auras',
             'aurez',
             'auriez',
             'aurions',
             'aurons',
             'auront',
             'aux',
             'avaient',
             'avais',
             'avait',
             'avec',
             'avez',
             'aviez',
             'avions',
             'avons',
             'ayant',
             'ayante',
             'ayantes',
             'ayants',
             'ayez',
             'ayons',
             'c',
             'ce',
             'ces',
             'd',
             'dans',
             'de',
             'des',
             'du',
             'elle',
             'en',
             'es',
             'est',
             'et',
             'eu',
             'eue',
             'eues',
             'eurent',
             'eus',
             'eusse',
             'eussent',
             'eusses',
             'eussiez',
             'eussions',
             'eut',
             'eux',
             'eûmes',
             'eût',
             'eûtes',
             'furent',
             'fus',
             'fusse',
             'fussent',
             'fusses',
             'fussiez',
             'fussions',
             'fut',
             'fûmes',
             'fût',
             'fûtes',
             'il',
             'j',
             'je',
             'l',
             'la',
             'le',
             'leur',
             'lui',
             'm',
             'ma',
             'mais',
             'me',
             'mes',
             'moi',
             'mon',
             'même',
             'n',
             'ne',
             'nos',
             'notre',
             'nous',
             'on',
             'ont',
             'ou',
             'par',
             'pas',
             'pour',
             'qu',
             'que',
             'qui',
             's',
             'sa',
             'se',
             'sera',
             'serai',
             'seraient',
             'serais',
             'serait',
             'seras',
             'serez', 
             'seriez',
             'serions',
             'serons',
             'seront',
             'ses',
             'soient',
             'sois',
             'soit',
             'sommes',
             'son',
             'sont',
             'soyez',
             'soyons',
             'suis',
             'sur',
             't',
             'ta',
             'te',
             'tes',
             'toi',
             'ton',
             'tu',
             'un',
             'une',
             'vos',
             'votre',
             'vous',
             'y',
             'à',
             'étaient',
             'étais',
             'était',
             'étant',
             'étante',
             'étantes',
             'étants',
             'étiez',
             'étions',
             'été',
             'étée',
             'étées',
             'étés',
             'êtes',
             ',',
             '(',
             ')',
             '[',
             ']',
             '-',
             '/',
             '+',
             '*',
             '{',
             '}',
             '!',
             ';',
             ':',
             '?',
             '.'
             }


In [10]:
#stop_words = set(open('stop_words.txt').read().split())
marques =  set(open('marques.txt').read().split())

In [11]:
stop_words_2 = stop_words.union(marques)
stop_words_2

{'!',
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '?',
 '[',
 ']',
 'a',
 'abeba',
 'abnet',
 'aboutbatteries',
 'abus',
 'acapulco',
 'achard',
 'actis',
 'acto',
 'addax',
 'adkalis',
 'aeg',
 'aegpowertools',
 'ai',
 'aie',
 'aient',
 'aies',
 'airnaturel',
 'ait',
 'akzonobel',
 'alaincompas',
 'alderacentertainmentgroup',
 'aldes',
 'algimouss',
 'alkern',
 'allett',
 'allgreen',
 'allister',
 'alpina',
 'amaco',
 'ambrogio',
 'ampack',
 'aqualux',
 'ardea',
 'ardex',
 'aria',
 'ariadeco',
 'ariafibres',
 'ariston',
 'aristonthermo',
 'aristonthermospa',
 'as',
 'astral',
 'astralpeinture',
 'astralpiscine',
 'atdv',
 'au',
 'aura',
 'aurai',
 'auraient',
 'aurais',
 'aurait',
 'auras',
 'aurez',
 'auriez',
 'aurions',
 'aurons',
 'auront',
 'autogyre',
 'aux',
 'avaient',
 'avais',
 'avait',
 'avec',
 'avez',
 'aviez',
 'avions',
 'avons',
 'axonfrance',
 'axton',
 'ayant',
 'ayante',
 'ayantes',
 'ayants',
 'ayez',
 'ayons',
 'balthazard',
 'balthazardetcotte',


In [12]:
# Clean stop words
df_ref = f_clean_stop_words(df_ref, ['ref_treetagger'], ['ref_clean2'], stop_words_2)

list_col = ['productfamily_seller_new', 'prodcategory4_seller_new', 'prodcategory3_seller_new', 'prodcategory2_seller_new', 'prodcategory1_seller_new', 'productseller_name_new']
list_new_col = ['productfamily_seller_clean2', 'prodcategory4_seller_clean2', 'prodcategory3_seller_clean2', 'prodcategory2_seller_clean2', 'prodcategory1_seller_clean2', 'productseller_name_clean2']
df_seller = f_clean_stop_words(df_seller, list_col, list_new_col, stop_words_2)


## Résultat

In [13]:
df_ref.head(5)

innoscape_ref_productfamily_seller  \
0                            Accessoires Gros oeuvre   
1                             Accessoires couverture   
3                                 Divers Gros oeuvre   
4                 Gouttière descente et accessoires    
5  Poudre béton chaux ciment enduit mortier ragréage   

                                      ref_treetagger  \
0                              accessoire gros œuvre   
1                              accessoire couverture   
3                                  divers gros œuvre   
4                   gouttiere descente et accessoire   
5  poudre beton chaux ciment enduire mortier ragr...   

                                          ref_clean2  
0                          [accessoire, gros, œuvre]  
1                           [accessoire, couverture]  
3                              [divers, œuvre, gros]  
4                  [descente, gouttiere, accessoire]  
5  [ciment, ragreage, beton, chaux, mortier, endu...

In [14]:
df_seller.head(5)

productfamily_seller prodcategory4_seller              prodcategory3_seller  \
0   Colles à carrelage   Colles à carrelage                Colles et adhésifs   
1                                                                               
2                                            Accessoires pour lave vaisselles   
3               Mastic               Mastic                Colles et adhésifs   
4                Cires                Cires   Peintures teintures et solvants   

                      prodcategory2_seller prodcategory1_seller  \
0                            Quincaillerie            Bricolage   
1                                                                 
2                              Accessoires  Gros électroménager   
3                            Quincaillerie            Bricolage   
4  Peintures outils et traitement des murs            Bricolage   

                                  productseller_name id_source  \
0  SikaCeram Xtra   Colle à carrelage intérieur e...    AMZ_FR   
1   Ariston 3200362 scaldabagno elettrico PRO 100 H3    AMZ_FR   
2  AEG Panier À Couverts Lave vaisselle Cage et H...    AMZ_FR   
3              Colle mastic pour joints d étanchéité    AMZ_FR   
4  Sikagard Protection Sol MAT   Imperméabilisant...    AMZ_FR   

       brand_seller productfamily_seller_new prodcategory4_seller_new  \
0     SIKAFRANCESAS        colle a carrelage        colle a carrelage   
1  ARISTONTHERMOSPA                                                     
2               AEG                                                     
3     SIKAFRANCESAS                   mastic                   mastic   
4     SIKAFRANCESAS                     cire                     cire   

         prodcategory3_seller_new             prodcategory2_seller_new  \
0                colle et adhesif                        quincaillerie   
1                                                                        
2  accessoire pour lave vaisselle                           accessoire   
3                colle et adhesif                        quincaillerie   
4    peinture teinture et solvant  peinture outil et traitement du mur   

  prodcategory1_seller_new                             productseller_name_new  \
0                bricolage  sikaceram xtra coller a carrelage interieur et...   
1                                     ariston scaldabagno elettrico pro heure   
2      gros electromenager  aeg panier a couvert laver vaisselle cage et h...   
3                bricolage               colle mastic pour joint d etancheite   
4                bricolage  sikagard protection sol mat impermeabiliser po...   

  productfamily_seller_clean2 prodcategory4_seller_clean2  \
0          [carrelage, colle]          [carrelage, colle]   
1                          []                          []   
2                          []                          []   
3                    [mastic]                    [mastic]   
4                      [cire]                      [cire]   

     prodcategory3_seller_clean2         prodcategory2_seller_clean2  \
0               [colle, adhesif]                     [quincaillerie]   
1                             []                                  []   
2  [accessoire, vaisselle, lave]                        [accessoire]   
3               [colle, adhesif]                     [quincaillerie]   
4  [peinture, teinture, solvant]  [traitement, outil, mur, peinture]   

  prodcategory1_seller_clean2  \
0                 [bricolage]   
1                          []   
2      [electromenager, gros]   
3                 [bricolage]   
4                 [bricolage]   

                           productseller_name_clean2  
0  [carrelage, sikaceram, exterieur, blanc, colle...  
1               [elettrico, heure, scaldabagno, pro]  
2  [handle, laver, cage, vaisselle, compartiment,...  
3                 [mastic, joint, etancheite, colle]  
4  [sol, mat, effet, sikagard, impermeabiliser, p...